# Solving the Poisson equation for Newtonian Potential in 3D

The Newtonian gravitational potential $\Phi(x)$ of a uniform density sphere in 3 dimensions satisfies, again, the equation 
$$-\Delta\Phi(x) = \rho(x)\quad x\in S^d_R,\quad d=3.$$

Here, $S^d_R$ is the sphere of radius $R$ in dimension $d$, and $\rho(x)$ is the mass density, taken to be constant, $\rho(x) = \rho_0$. 

Like the 2D problem, the problem in 3 dimensions has an analytic solution [See Cohl and Palmer, *Fourier and Gegenbauer Expansions for a Fundamental Solution of Laplace's Equation in Hyperspherical Geometry*] 

In particular, for uniform density $\rho = \rho_0>0$, and radius $R = r_0>0$, the exact solution is

$$ \Phi(x) :=\cases{-\frac{\rho_0}{6}\big(3r_0^2 - r^2\big),\quad r\in[0,r_0], \\
\frac{\rho_0 r_0^3}{3r},\quad r\in(r_0,\infty).}$$

This formula says that $\Phi(r_0)=\frac{\rho_0 r_0^2}{3}$ on the boundary. We use this as the boundary data for our numerical solver. 

In [1]:
########################################################################################################
################################## MESHING #############################################################
########################################################################################################
import dolfinx
from mpi4py import MPI
#mesh = dolfinx.mesh.create_unit_square(MPI.COMM_WORLD, 10, 10)

import gmsh
gmsh.initialize()

sphere = gmsh.model.occ.addSphere(0, 0, 0, 1)
#gmsh.model.occ.addCurveLoop([ellipse], 5)
#membrane = gmsh.model.occ.addPlaneSurface([5])
gmsh.model.occ.synchronize()

gdim = 3
gmsh.model.addPhysicalGroup(gdim, [sphere], 1)

gmsh.option.setNumber("Mesh.CharacteristicLengthMin",0.1)
gmsh.option.setNumber("Mesh.CharacteristicLengthMax",0.1)
gmsh.model.mesh.generate(gdim)

from dolfinx.io import gmshio

gmsh_model_rank = 0
mesh_comm = MPI.COMM_WORLD

gmsh.write("mesh3D.msh")


domain, cell_markers, facet_markers = gmshio.model_to_mesh(gmsh.model, mesh_comm, gmsh_model_rank, gdim=3)

########################################################################################################
################################## DEFINE EXACT SOLUTION ###############################################
########################################################################################################
from dolfinx import fem, mesh, fem, io, nls, log
import ufl
import numpy
from petsc4py.PETSc import ScalarType

#define the values of mass density, radius, and log of radius, and create constant functions on 
#the mesh for these values
rho0_0 = 12 #Mass density
R0_0 = 1.0 #Radius of disc
lnR0_0 = numpy.log(R0_0) #log of disc radius

x = ufl.SpatialCoordinate(domain)
rho0 = fem.Constant(domain, ScalarType(rho0_0))
R0 = fem.Constant(domain, ScalarType(R0_0)) 
lnR0 = fem.Constant(domain, ScalarType(lnR0_0))

#p is the exact analytic solution
p = (rho0 / 6) * (3 * R0**2 - (x[0]**2 + x[1]**2 + x[2]**2))

########################################################################################################
################################## DEFINE BOUNDARY CONDITION ###########################################
########################################################################################################
#note that u_ufl is the same function as p, but is specified using, e.g., R0_0 instead of R0
u_ufl = (rho0_0 / 6) * (3 * R0_0**2 - (x[0]**2 + x[1]**2 + x[2]**2))
V = fem.FunctionSpace(domain, ("CG", 1))
u_exact = lambda x: eval(str(u_ufl))
u_D = fem.Function(V)
u_D.interpolate(u_exact)
fdim = domain.topology.dim - 1
boundary_facets = mesh.locate_entities_boundary(domain, fdim, lambda x: numpy.full(x.shape[1], True, dtype=bool))

bc = fem.dirichletbc(u_D, fem.locate_dofs_topological(V, fdim, boundary_facets))

########################################################################################################
################################## ALTERNATIVE SPECIFICATION OF BOUNDARY DATA ##########################
########################################################################################################
#import numpy as np
#def on_boundary(x):
#    return np.isclose(np.sqrt((x[0])**2 + x[1]**2), 1) #must be changed accordingly for elliptic boundary
#boundary_dofs = fem.locate_dofs_geometrical(V, on_boundary)

#bc = fem.dirichletbc(p, boundary_dofs, V)

########################################################################################################
################################## SET UP AND SOLVE VARIATIONAL PROBLEM ################################
########################################################################################################
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = rho0 * v * ufl.dx
problem = fem.petsc.LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

########################################################################################################
################################## PLOT SOLUTION USING PYVISTA #########################################
########################################################################################################
from dolfinx.plot import create_vtk_mesh
import pyvista
pyvista.set_jupyter_backend("pythreejs")

# Extract topology from mesh and create pyvista mesh
topology, cell_types, x = create_vtk_mesh(V)
grid = pyvista.UnstructuredGrid(topology, cell_types, x)

# Set deflection values and add it to plotter
grid.point_data["u"] = uh.x.array
warped = grid.warp_by_scalar("u", factor=.25)

plotter = pyvista.Plotter()
plotter.add_mesh(grid, style = 'points', point_size = 25, show_edges = True, opacity = .5)

if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    pyvista.start_xvfb()
    plotter.screenshot("deflection.png")

Info    : Meshing 1D...
Info    : [ 40%] Meshing curve 2 (Circle)
Info    : Done meshing 1D (Wall 0.000153s, CPU 0.000112s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Sphere, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0922597s, CPU 0.078235s)
Info    : Meshing 3D...
Info    : 3D Meshing 1 volume with 1 connected component
Info    : Tetrahedrizing 1578 nodes...
Info    : Done tetrahedrizing 1586 nodes (Wall 0.0210445s, CPU 0.021076s)
Info    : Reconstructing mesh...
Info    :  - Creating surface mesh
Info    :  - Identifying boundary edges
Info    :  - Recovering boundary
Info    : Done reconstructing mesh (Wall 0.0455781s, CPU 0.045185s)
Info    : Found volume 1
Info    : It. 0 - 0 nodes created - worst tet radius 10.7044 (nodes removed 0 0)
Info    : It. 500 - 500 nodes created - worst tet radius 1.60862 (nodes removed 0 0)
Info    : It. 1000 - 1000 nodes created - worst tet radius 1.31171 (nodes removed 0 0)
Info    : It. 1500 - 1500 nodes created - worst tet radiu

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

In [6]:
########################################################################################################
################################## PLOT SOLUTION USING PYVISTA #########################################
########################################################################################################
from dolfinx.plot import create_vtk_mesh
import pyvista
pyvista.set_jupyter_backend("pythreejs")

# Extract topology from mesh and create pyvista mesh
topology, cell_types, x = create_vtk_mesh(V)
grid = pyvista.UnstructuredGrid(topology, cell_types, x)

# Set deflection values and add it to plotter
grid.point_data["u"] = uh.x.array
warped = grid.warp_by_scalar("u", factor=.125)

plotter = pyvista.Plotter()
plotter.add_mesh(grid, style = "wireframe", color = 'y')
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    pyvista.start_xvfb()
    plotter.screenshot("deflection.png")

2022-12-14 11:41:50.881 ( 179.472s) [        ABFDB740]    vtkExtractEdges.cxx:435   INFO| Executing edge extractor: points are renumbered
2022-12-14 11:41:50.883 ( 179.474s) [        ABFDB740]    vtkExtractEdges.cxx:551   INFO| Created 4728 edges


Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

In [2]:
import ufl
import numpy as np
from mpi4py import MPI
from petsc4py import PETSc
from dolfinx.fem import (Expression, Function, FunctionSpace,
                         assemble_scalar, dirichletbc, form, locate_dofs_topological)
from dolfinx.fem.petsc import LinearProblem
from dolfinx.mesh import create_unit_square, locate_entities_boundary
from ufl import SpatialCoordinate, TestFunction, TrialFunction, div, dot, dx, grad, inner

def error_infinity(u_h, u_ex):
    # Interpolate exact solution, special handling if exact solution
    # is a ufl expression or a python lambda function
    comm = u_h.function_space.mesh.comm
    u_ex_V = Function(u_h.function_space)
    if isinstance(u_ex, ufl.core.expr.Expr):
        u_expr = Expression(u_ex, u_h.function_space.element.interpolation_points)
        u_ex_V.interpolate(u_expr)
    else:
        u_ex_V.interpolate(u_ex)
    # Compute infinity norm, furst local to process, then gather the max
    # value over all processes
    error_max_local = np.max(np.abs((u_h.x.array-u_ex_V.x.array)/u_ex_V.x.array))
    error_max = comm.allreduce(error_max_local, op=MPI.MAX)
    return error_max

In [3]:
error_infinity(uh, u_exact)

0.001961798312527939